# ElectroCarDiogram (ECG) clasifyer whit Dynamic Time Wrapping (DTW)

The Imports

In [1]:
# nessesery imports
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Loading the datasets

In [2]:
# Data loading and preprocessing
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values

In [3]:
#werifying the the size of the data set
dataframe.shape

(4998, 141)

In [4]:
# lock at th data
dataframe.tail()

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
4993,0.608558,-0.335651,-0.990948,-1.784153,-2.626145,-2.957065,-2.931897,-2.664816,-2.090137,-1.461841,...,1.757705,2.291923,2.704595,2.451519,2.017396,1.704358,1.688542,1.629593,1.342651,0.0
4994,-2.060402,-2.860116,-3.405074,-3.748719,-3.513561,-3.006545,-2.234850,-1.593270,-1.075279,-0.976047,...,1.388947,2.079675,2.433375,2.159484,1.819747,1.534767,1.696818,1.483832,1.047612,0.0
4995,-1.122969,-2.252925,-2.867628,-3.358605,-3.167849,-2.638360,-1.664162,-0.935655,-0.866953,-0.645363,...,-0.472419,-1.310147,-2.029521,-3.221294,-4.176790,-4.009720,-2.874136,-2.008369,-1.808334,0.0
4996,-0.547705,-1.889545,-2.839779,-3.457912,-3.929149,-3.966026,-3.492560,-2.695270,-1.849691,-1.374321,...,1.258419,1.907530,2.280888,1.895242,1.437702,1.193433,1.261335,1.150449,0.804932,0.0
4997,-1.351779,-2.209006,-2.520225,-3.061475,-3.065141,-3.030739,-2.622720,-2.044092,-1.295874,-0.733839,...,-1.512234,-2.076075,-2.586042,-3.322799,-3.627311,-3.437038,-2.260023,-1.577823,-0.684531,0.0


In [5]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,1.0
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1.0
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1.0
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1.0
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1.0


In [6]:
# Extract labels and data points
# The last element contains the labels
labels = raw_data[:, -1]

# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=0
)

separating normal and abnormal data

In [7]:
train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

In [8]:
normal_train_data.shape

(2333, 140)

In [9]:
print(normal_train_data)

[[-3.0539033  -4.4028867  -4.1376094  ...  0.56791308  1.0003015
  -0.1913969 ]
 [ 1.7587573   0.48943753 -1.5222317  ...  1.5104879   1.0361705
   1.6157525 ]
 [-0.94521791 -2.560839   -3.4157185  ... -1.0552072  -0.46383879
  -1.2705295 ]
 ...
 [-0.57823582 -2.8430014  -3.623841   ...  0.37186676  1.0697144
  -0.60282055]
 [-1.0504592  -3.1380982  -3.7998881  ...  1.6135197   2.2570463
   0.34046592]
 [-0.85346093 -3.7171244  -4.6348932  ...  1.5531752   1.8897122
  -0.67465939]]


In [10]:
anomalous_train_data.shape

(1665, 140)

In [11]:
# picing a random normal data index to use in the referance data
random.randint(0,2333)

676

In [12]:
# to pic an radom abnormal data index to use in the referance data
random.randint(0,1665)

218

Defining th referance signals that that the DTW uses as referance data to clasify the test data on

In [13]:
# after running the random number for normal and abnor mal twise we haw the referance data to use
# defining the referance test ecg signals
# Example usage with loop over test signals
# differnet number of referance data would efect the acuracy scor of the prediction
ECGs = {
    'Normal Ref ECG 1': normal_train_data[2181],
    'Abnormal Ref ECG 1': anomalous_train_data[1491],
    'Normal Ref ECG 2': normal_train_data[1198],
    'Abnormal Ref ECG 2': anomalous_train_data[95],
    'Normal Ref ECG 3': normal_train_data[1742],
    'Abnormal Ref ECG 3': anomalous_train_data[153],
}

The defining of the dtw algorithem an the matrix

In [14]:
# define the dtw and the matrix
def dtw(x, y, window):
    n, m = len(x), len(y)
    window = max(window, abs(n - m))
    dtw_matrix = np.full((n+1, m+1), np.inf)
    dtw_matrix[0, 0] = 0

    for i in range(1, n+1):
        for j in range(max(1, i-window), min(m+1, i+window)):
            cost = abs(x[i-1] - y[j-1])
            last_min = np.min(dtw_matrix[i-1:i, j-1:j+1])
            dtw_matrix[i, j] = cost + last_min

    return dtw_matrix[-1, -1]

In [15]:
# tiesting different window sizes and calculate acuracy and run time
window_sizes = [1, 2, 3, 4, 5, 10] # List of window sizes to test
for window in window_sizes:
    print(f"Testing window size: {window}")
    num_correct = 0
    num_incorrect = 0
    start_time = time.time() # start time for the current window size
    for i in range(len(normal_test_data)):
        test_signal = normal_test_data[i]
        scores = []
        for ecg in ECGs:
            score = dtw(test_signal, ECGs[ecg], window=window)
            scores.append(score)
        predicted_ecg = list(ECGs.keys())[np.argmin(scores)]
        if 'Abnormal' in predicted_ecg:
            num_incorrect += 1
        else:
            num_correct += 1

    accuracy = num_correct / len(normal_test_data)
    end_time = time.time() # end time for the current window size
    print(f"Accuracy: {accuracy}")
    print(f"Run time for window size {window}: {end_time - start_time} seconds") # individual run time for the current window size


Testing window size: 1
Accuracy: 0.9641638225255973
Run time for window size 1: 14.438557147979736 seconds
Testing window size: 2
Accuracy: 0.9641638225255973
Run time for window size 2: 21.658792972564697 seconds
Testing window size: 3
Accuracy: 0.9641638225255973
Run time for window size 3: 17.335017204284668 seconds
Testing window size: 4
Accuracy: 0.9641638225255973
Run time for window size 4: 23.733670473098755 seconds
Testing window size: 5
Accuracy: 0.9641638225255973
Run time for window size 5: 28.581380605697632 seconds
Testing window size: 10
Accuracy: 0.9641638225255973
Run time for window size 10: 58.46135878562927 seconds


In [16]:
#luoping through all the normal test data and calculate the acuracy of te clasification and calculating the total executing tim

start_time = time.time()
num_correct = 0
num_incorrect = 0
for i in range(len(normal_test_data)):
    test_signal = normal_test_data[i]
    scores = []
    for ecg in ECGs:
        score = dtw(test_signal, ECGs[ecg], window=1)
        scores.append(score)
    predicted_ecg = list(ECGs.keys())[np.argmin(scores)]
    if 'Abnormal' in predicted_ecg:
        num_incorrect += 1
    else:
        num_correct += 1

accuracy = num_correct / len(normal_test_data)
print(f"Accuracy: {accuracy}")

end_time = time.time()
total_time = end_time - start_time

print(f"Total execution time: {total_time} seconds")
print(f"total test data:", len(normal_test_data))


Accuracy: 0.9641638225255973
Total execution time: 5.6943724155426025 seconds
total test data: 586


In [17]:
# luoping through all the abnormall test data and calculate the acuracy of te clasification and calculating the total executing tim

start_time = time.time()
num_correct = 0
num_incorrect = 0
for i in range(len(anomalous_test_data)):
    test_signal = anomalous_test_data[i]
    scores = []
    for ecg in ECGs:
        score = dtw(test_signal, ECGs[ecg], window=1)
        scores.append(score)
    predicted_ecg = list(ECGs.keys())[np.argmin(scores)]
    if 'Abnormal' in predicted_ecg:
        num_correct += 1
    else:
        num_incorrect += 1

accuracy = num_correct / len(anomalous_test_data)
print(f"Accuracy: {accuracy}")

end_time = time.time()
total_time = end_time - start_time

print(f"Total execution time: {total_time} seconds")
print(f"total test data:", len(anomalous_test_data))


Accuracy: 0.9758454106280193
Total execution time: 5.244083642959595 seconds
total test data: 414
